Archive:  /content/SmartFolio.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/SmartFolio.zip or
        /content/SmartFolio.zip.zip, and cannot find /content/SmartFolio.zip.ZIP, period.


In [13]:
cd /content

/content


In [14]:
!rm -rf SmartFolio

In [16]:
import os, subprocess, sys, textwrap, json

# Check GPU
gpu = !nvidia-smi -L
print("GPU detected:" if gpu else "No GPU detected.", gpu if gpu else "")

# Clone repo fresh
REPO_URL = "https://github.com/TakshayBansal/SmartFolio.git"
WORKDIR = "/content/SmartFolio"
if os.path.isdir(WORKDIR):
    !rm -rf "{WORKDIR}"
!git clone "{REPO_URL}" "{WORKDIR}"
%cd "{WORKDIR}"
!git rev-parse --short HEAD

GPU detected: ['GPU 0: Tesla T4 (UUID: GPU-2e95d582-a477-2d2e-5976-c6618470b9b7)']
Cloning into '/content/SmartFolio'...
remote: Enumerating objects: 3528, done.
remote: Total 3528 (delta 0), reused 0 (delta 0), pack-reused 3528 (from 1)
Receiving objects: 100% (3528/3528), 161.06 MiB | 23.03 MiB/s, done.
Resolving deltas: 100% (1663/1663), done.
Updating files: 100% (5191/5191), done.
/content/SmartFolio
237a26b


In [ ]:
# cd /content/SmartFolio/

/content/SmartFolio


In [7]:
import yfinance as yf
df = yf.download("MSFT", start="2017-01-01", end="2017-04-30")
df.head()

/tmp/ipython-input-3565612260.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("MSFT", start="2017-01-01", end="2017-04-30")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
Date,,,,,
2017-01-03,56.404678,56.639020,55.999083,56.593955,20694100
2017-01-04,56.152325,56.557920,55.990087,56.314563,21340000
2017-01-05,56.152325,56.476801,55.908967,56.053179,24876000
2017-01-06,56.639030,56.918441,55.917974,56.152316,19922900
2017-01-09,56.458763,56.855347,56.368632,56.566921,20382700


In [6]:
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 5.7 MB/s eta 0:00:00


In [7]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.5 MB/s eta 0:00:00


In [13]:
import os

base = "/content/SmartFolio/dataset"
os.makedirs(base, exist_ok=True)
os.makedirs(os.path.join(base, "index_data"), exist_ok=True)
# Example market path (hs300). Adjust if you’ll run other markets.
os.makedirs(os.path.join(base, "data_train_predict_hs300", "1_hy"), exist_ok=True)

print("Created placeholders under:", base)
print("Now upload your data to these folders via the Colab file browser (left pane).")

Created placeholders under: /content/SmartFolio/dataset
Now upload your data to these folders via the Colab file browser (left pane).


In [20]:
# Compatibility shim: robustify IRL process_data to handle samples without pyg_data/industry matrices
import importlib
import torch
from torch_geometric.data import Data
import trainer.irl_trainer as irl

def compat_process_data(data_dict, device="cuda:0"):
    corr = data_dict['corr'].to(device).squeeze()
    ts_features = data_dict.get('ts_features', torch.zeros(1, device=device)).to(device).squeeze()
    features = data_dict['features'].to(device).squeeze()

    # Infer number of stocks
    N = features.shape[-2] if features.dim() == 3 else features.shape[0]

    # Industry/momentum/reversal graphs; fallbacks if missing
    ind = data_dict.get('industry_matrix')
    pos = data_dict.get('pos_matrix')
    neg = data_dict.get('neg_matrix')
    if ind is None:
        ind = (corr > 0).float()  # simple fallback from corr
    if pos is None:
        pos = torch.zeros_like(ind)
    if neg is None:
        neg = torch.zeros_like(ind)
    ind, pos, neg = ind.to(device).squeeze(), pos.to(device).squeeze(), neg.to(device).squeeze()

    # Graph object; build if missing
    pyg_data = data_dict.get('pyg_data')
    if pyg_data is None:
        x = features if features.dim() == 2 else features[0]  # take the first step if [T, N, F]
        edge_index = torch.triu_indices(N, N, offset=1)
        edge_attr = ind[edge_index[0], edge_index[1]] if ind.dim() == 2 else None
        pyg_data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    else:
        pyg_data = pyg_data.to(device)

    labels = data_dict['labels'].to(device).squeeze()
    # Default mask if missing
    mask = data_dict.get('mask', [True] * (labels.shape[-1] if labels.dim() > 1 else N))
    return corr, ts_features, features, ind, pos, neg, labels, pyg_data, mask


irl.process_data = compat_process_data
importlib.reload(irl)
print("IRL process_data patched for backward compatibility.")

IRL process_data patched for backward compatibility.


In [22]:
import os
import glob
import pickle
from tqdm import tqdm

import torch
from torch_geometric.data import Data

PKL_DIR = "/content/SmartFolio/dataset/data_train_predict_hs300/1_hy"

def pick_time0(t):
    # Return tensor squeezed to 2D node x feat if possible, taking time step 0 when 3D
    if isinstance(t, torch.Tensor):
        if t.dim() >= 3:
            return t[0]
        return t
    return t

def ensure_float32(t):
    if isinstance(t, torch.Tensor):
        return t.to(dtype=torch.float32, device="cpu")
    return t

def build_pyg_data_from_sample(sample):
    """
    Build torch_geometric.data.Data for a sample dict.
    - x: node features (N, F), taken from 'features' (time step 0 if 3D)
    - edge_index: upper-triangular undirected edges
    - edge_attr: from industry_matrix if available, else from corr
    """
    if "features" not in sample:
        raise ValueError("Missing 'features' tensor in sample; cannot construct pyg_data.")
    features = ensure_float32(sample["features"])
    x = pick_time0(features)  # (N, F) expected
    if not isinstance(x, torch.Tensor) or x.dim() != 2:
        raise ValueError(f"Could not derive 2D node feature matrix from 'features'; got shape {getattr(x, 'shape', None)}")

    N = x.shape[0]
    edge_index = torch.triu_indices(N, N, offset=1)  # upper-triangular without diagonal

    # Choose source for edge_attr
    edge_attr_src = None
    if "industry_matrix" in sample:
        ind = ensure_float32(sample["industry_matrix"])
        ind2d = pick_time0(ind)  # (N, N)
        if isinstance(ind2d, torch.Tensor) and ind2d.dim() == 2 and ind2d.shape[0] == N and ind2d.shape[1] == N:
            edge_attr_src = ind2d
    if edge_attr_src is None and "corr" in sample:
        corr = ensure_float32(sample["corr"])
        corr2d = pick_time0(corr)
        if isinstance(corr2d, torch.Tensor) and corr2d.dim() == 2 and corr2d.shape[0] == N and corr2d.shape[1] == N:
            edge_attr_src = corr2d

    if edge_attr_src is not None:
        edge_attr = edge_attr_src[edge_index[0], edge_index[1]].to(dtype=torch.float32, device="cpu")
    else:
        # If neither matrix found or shape mismatch, set zeros
        edge_attr = torch.zeros(edge_index.shape[1], dtype=torch.float32)

    pyg_data = Data(x=x.cpu(), edge_index=edge_index.cpu(), edge_attr=edge_attr.cpu())
    return pyg_data

def main():
    files = sorted(glob.glob(os.path.join(PKL_DIR, "*.pkl")))
    print(f"Found {len(files)} pkl files under: {PKL_DIR}")
    fixed, skipped, failed = 0, 0, 0

    for fp in tqdm(files):
        try:
            with open(fp, "rb") as f:
                sample = pickle.load(f)

            if isinstance(sample, dict) and "pyg_data" in sample and sample["pyg_data"] is not None:
                skipped += 1
                continue

            pyg_data = build_pyg_data_from_sample(sample)
            sample["pyg_data"] = pyg_data

            # Backup before overwrite
            bak = fp + ".bak"
            if not os.path.exists(bak):
                with open(bak, "wb") as f:
                    pickle.dump(sample, f)

            # Write updated sample
            with open(fp, "wb") as f:
                pickle.dump(sample, f)

            fixed += 1
        except Exception as e:
            failed += 1
            print(f"\n[ERROR] {fp}: {e}")

    print(f"Done. Fixed: {fixed}, Skipped (already had pyg_data): {skipped}, Failed: {failed}")

if __name__ == "__main__":
    main()

Found 1697 pkl files under: /content/SmartFolio/dataset/data_train_predict_hs300/1_hy


  2%|▏         | 31/1697 [00:00<00:05, 287.39it/s]


[ERROR] /content/SmartFolio/dataset/data_train_predict_hs300/1_hy/2018-01-03.pkl: Could not derive 2D node feature matrix from 'features'; got shape torch.Size([0])

[ERROR] /content/SmartFolio/dataset/data_train_predict_hs300/1_hy/2018-01-04.pkl: Could not derive 2D node feature matrix from 'features'; got shape torch.Size([0])

[ERROR] /content/SmartFolio/dataset/data_train_predict_hs300/1_hy/2018-01-05.pkl: Could not derive 2D node feature matrix from 'features'; got shape torch.Size([0])

[ERROR] /content/SmartFolio/dataset/data_train_predict_hs300/1_hy/2018-01-08.pkl: Could not derive 2D node feature matrix from 'features'; got shape torch.Size([0])

[ERROR] /content/SmartFolio/dataset/data_train_predict_hs300/1_hy/2018-01-09.pkl: Could not derive 2D node feature matrix from 'features'; got shape torch.Size([0])

[ERROR] /content/SmartFolio/dataset/data_train_predict_hs300/1_hy/2018-01-10.pkl: Could not derive 2D node feature matrix from 'features'; got shape torch.Size([0])

[ER

100%|██████████| 1697/1697 [00:07<00:00, 241.53it/s]

Done. Fixed: 1678, Skipped (already had pyg_data): 0, Failed: 19


In [23]:
import pickle

file_path = "/content/SmartFolio/dataset/data_train_predict_hs300/1_hy/2018-03-29.pkl"

try:
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    print("Keys in the pickle file:", data.keys())
except FileNotFoundError:
    print(f"Error: The file {file_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Keys in the pickle file: dict_keys(['corr', 'ts_features', 'features', 'industry_matrix', 'pos_matrix', 'neg_matrix', 'labels', 'mask', 'pyg_data'])


In [8]:
!pip install shimmy>=2.0

In [32]:
import os
import glob
import pickle
import numpy as np
import torch

ROOT = "/content/SmartFolio"
PKL_DIR = f"{ROOT}/dataset_default/data_train_predict_hs300/1_hy"
TARGET_DIR = f"{ROOT}/dataset_default/data_train_predict_hs300"
TARGET_FILE = f"{TARGET_DIR}/industry.npy"

def ensure_dataset_default_symlink():
    ds = f"{ROOT}/dataset"
    dsd = f"{ROOT}/dataset_default"
    if not os.path.exists(dsd):
        try:
            os.symlink(ds, dsd)
            print(f"Created symlink: {dsd} -> {ds}")
        except FileExistsError:
            pass

def extract_industry_matrix_from_sample(pkl_path):
    with open(pkl_path, "rb") as f:
        sample = pickle.load(f)
    # Prefer industry_matrix in the sample
    if "industry_matrix" in sample:
        mat = sample["industry_matrix"]
        if isinstance(mat, torch.Tensor):
            mat = mat.detach().cpu().numpy()
        else:
            mat = np.asarray(mat)
        if mat.ndim == 3:
            # If it has a time dimension, take the first
            mat = mat[0]
        if mat.ndim != 2 or mat.shape[0] != mat.shape[1]:
            raise ValueError(f"industry_matrix not square: shape={mat.shape}")
        return mat
    # Fallback: derive from corr (same-industry unknown; here we just use identity)
    if "corr" in sample:
        corr = sample["corr"]
        if isinstance(corr, torch.Tensor):
            corr = corr.detach().cpu().numpy()
        else:
            corr = np.asarray(corr)
        if corr.ndim == 3:
            corr = corr[0]
        n = corr.shape[0]
        print("No industry_matrix found; using identity as fallback.")
        return np.eye(n, dtype=np.float32)
    raise KeyError("Neither 'industry_matrix' nor 'corr' present in sample.")

def main():
    ensure_dataset_default_symlink()
    os.makedirs(TARGET_DIR, exist_ok=True)

    files = sorted(glob.glob(os.path.join(PKL_DIR, "*.pkl")))
    if not files:
        raise FileNotFoundError(f"No .pkl files found under {PKL_DIR}")

    # Use the first file to extract a representative industry matrix
    mat = extract_industry_matrix_from_sample(files[0])

    np.save(TARGET_FILE, mat.astype(np.float32))
    print(f"Saved industry matrix to: {TARGET_FILE} with shape {mat.shape}")

if __name__ == "__main__":
    main()

Saved industry matrix to: /content/SmartFolio/dataset_default/data_train_predict_hs300/industry.npy with shape (102, 102)


In [19]:
%cd /content/SmartFolio
!python main.py

/content/SmartFolio
True
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
2025-10-28 17:37:26.855706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761673046.875758   16899 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761673046.881790   16899 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761673046.896788   16899 computation

In [35]:
import os
import glob
import pickle
import numpy as np
import pandas as pd
from datetime import datetime

ROOT = "/content/SmartFolio"
PKL_DIR = f"{ROOT}/dataset_default/data_train_predict_hs300/1_hy"

# IRL code reads "../dataset/index_data/...", which resolves to "/content/dataset/index_data"
PARENT_DATASET_DIR = os.path.join(os.path.dirname(ROOT), "dataset", "index_data")  # /content/dataset/index_data
LOCAL_DATASET_DIR = os.path.join(ROOT, "dataset", "index_data")                     # /content/SmartFolio/dataset/index_data

os.makedirs(PARENT_DATASET_DIR, exist_ok=True)
os.makedirs(LOCAL_DATASET_DIR, exist_ok=True)

def parse_date_from_filename(path):
    # Files are named like "YYYY-MM-DD.pkl"
    base = os.path.basename(path)
    ds = base[:10]
    try:
        # validate format
        datetime.strptime(ds, "%Y-%m-%d")
        return ds
    except ValueError:
        return None

def mean_return_from_labels(sample):
    labels = sample.get("labels", None)
    if labels is None:
        return 0.0
    try:
        arr = labels.detach().cpu().numpy() if hasattr(labels, "detach") else np.asarray(labels)
        arr = np.nan_to_num(arr, nan=0.0)
        if arr.ndim == 0:
            return float(arr)
        return float(np.mean(arr))
    except Exception:
        return 0.0

def main():
    files = sorted(glob.glob(os.path.join(PKL_DIR, "*.pkl")))
    if not files:
        raise FileNotFoundError(f"No PKL files found under {PKL_DIR}")

    rows = []
    for fp in files:
        dt = parse_date_from_filename(fp)
        if not dt:
            continue
        with open(fp, "rb") as f:
            sample = pickle.load(f)
        ew_ret = mean_return_from_labels(sample)
        rows.append((dt, ew_ret))

    if not rows:
        raise RuntimeError("No valid dates parsed from PKL filenames; cannot build index CSV.")

    df = pd.DataFrame(rows, columns=["datetime", "daily_return"]).sort_values("datetime")

    # Keep only 2024 for this file (as IRL reads *_index_2024.csv)
    df["year"] = df["datetime"].str.slice(0, 4)
    df_2024 = df[df["year"] == "2024"].drop(columns=["year"])
    if df_2024.empty:
        print("Warning: No 2024 dates found; writing full date range instead.")
        df_2024 = df.drop(columns=["year"])

    for target_dir in (PARENT_DATASET_DIR, LOCAL_DATASET_DIR):
        out = os.path.join(target_dir, "hs300_index_2024.csv")
        df_2024.to_csv(out, index=False)
        print(f"Wrote: {out}  rows={len(df_2024)}")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


Wrote: /content/dataset/index_data/hs300_index_2024.csv  rows=241
Wrote: /content/SmartFolio/dataset/index_data/hs300_index_2024.csv  rows=241


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [38]:
import os
from google.colab import files

# Define the directory to zip
directory_to_zip = '/content/SmartFolio'
# Define the name for the zipped file
zipped_filename = 'SmartFolio.zip'

# Zip the directory
!zip -r "{zipped_filename}" "{directory_to_zip}"

# Download the zipped file
files.download(zipped_filename)

Streaming output truncated to the last 5000 lines.
  adding: content/SmartFolio/dataset_default/data_train_predict_hs300/1_hy/2023-09-21.pkl (deflated 61%)
  adding: content/SmartFolio/dataset_default/data_train_predict_hs300/1_hy/2018-08-27.pkl (deflated 61%)
  adding: content/SmartFolio/dataset_default/data_train_predict_hs300/1_hy/2022-03-16.pkl.bak (deflated 62%)
  adding: content/SmartFolio/dataset_default/data_train_predict_hs300/1_hy/2021-01-29.pkl.bak (deflated 60%)
  adding: content/SmartFolio/dataset_default/data_train_predict_hs300/1_hy/2022-09-14.pkl (deflated 62%)
  adding: content/SmartFolio/dataset_default/data_train_predict_hs300/1_hy/2024-03-18.pkl (deflated 60%)
  adding: content/SmartFolio/dataset_default/data_train_predict_hs300/1_hy/2021-09-23.pkl.bak (deflated 60%)
  adding: content/SmartFolio/dataset_default/data_train_predict_hs300/1_hy/2020-09-01.pkl (deflated 61%)
  adding: content/SmartFolio/dataset_default/data_train_predict_hs300/1_hy/2020-07-24.pkl (deflat

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>